In [1]:
!pip install opencv-python --user

In [9]:
import cv2 as cv
import math
import time
import argparse
import sys
sys.argv=['']
del sys

#conf_threshold means if the accuracy of recognizing the correct result is greater than 70% then only it will the result be displayed

def getFaceBox(net, frame, conf_threshold=0.7): 
    frameOpencvDnn = frame.copy() #inputting the frame
    frameHeight = frameOpencvDnn.shape[0] 
    frameWidth = frameOpencvDnn.shape[1]
    #creating a 4 dimesional array of image
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)
    #by default scale factor is 1.0,resizing the frame to 300X300,mean subtractoin value,swaprb is true to convert BGR to RGB,crop is false, we want the actaul size of the image
    
    net.setInput(blob)
    #after preprocessing the image it is passed to neural network
    
    detections = net.forward()
    #net.forward will gives the output after processing
    
    bboxes = []
    #loop for drawing rectangle on the identified face
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes
parser = argparse.ArgumentParser(description='Use this script to run age and gender recognition using OpenCV.')
parser.add_argument('--input', help='Path to input image or video file. Skip this argument to capture frames from a camera.')

args = parser.parse_args()

faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-80)']
genderList = ['Male', 'Female']

# Load network
ageNet = cv.dnn.readNet(ageModel, ageProto)
genderNet = cv.dnn.readNet(genderModel, genderProto)
faceNet = cv.dnn.readNet(faceModel, faceProto)

# Open a video file or an image file or a camera stream
cap = cv.VideoCapture(args.input if args.input else 0)
padding = 20

while cv.waitKey(1) < 0: #while the video is still on/or you haven't exit
    # Read frame
    t = time.time()
    hasFrame, frame = cap.read()
    if not hasFrame: #if there is not frame break the video
        cv.waitKey()
        break

    frameFace, bboxes = getFaceBox(faceNet, frame)
    if not bboxes:
       print("No face Detected, Checking next frame")
#      continue

    for bbox in bboxes:
        # print(bbox)
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        #      for gender preediction
        blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        # print("Gender Output : {}".format(genderPreds))
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        #      for age preediction
        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv.putText(frameFace, label, (bbox[0], bbox[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv.LINE_AA)
        cv.imshow("Age Gender", frameFace)

Gender : Male, conf = 0.948
Age : (25-32), conf = 0.990
Gender : Male, conf = 0.966
Age : (25-32), conf = 0.962
Gender : Male, conf = 0.971
Age : (25-32), conf = 0.941
Gender : Male, conf = 0.919
Age : (25-32), conf = 0.974
Gender : Male, conf = 0.827
Age : (25-32), conf = 0.966
Gender : Male, conf = 0.880
Age : (25-32), conf = 0.965
Gender : Male, conf = 0.949
Age : (25-32), conf = 0.946
Gender : Male, conf = 0.936
Age : (25-32), conf = 0.753
Gender : Male, conf = 0.939
Age : (25-32), conf = 0.927
Gender : Male, conf = 0.898
Age : (25-32), conf = 0.946
Gender : Male, conf = 0.962
Age : (25-32), conf = 0.937
Gender : Male, conf = 0.914
Age : (25-32), conf = 0.966
Gender : Male, conf = 0.978
Age : (25-32), conf = 0.972
Gender : Male, conf = 0.944
Age : (25-32), conf = 0.974
Gender : Male, conf = 0.969
Age : (25-32), conf = 0.979
Gender : Male, conf = 0.961
Age : (25-32), conf = 0.987
Gender : Male, conf = 0.943
Age : (25-32), conf = 0.987
Gender : Male, conf = 0.973
Age : (25-32), conf 

KeyboardInterrupt: 